<a href="https://colab.research.google.com/github/KSY1526/SSUDA_LOTTEs/blob/main/LPOINT_Final_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# '''
# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지"); 
#     document.querySelector("colab-toolbar-button#connect").click() 
# }
# setInterval(ClickConnect, 60 * 1000)
# '''

In [2]:
!pip install fastpivot
!pip install pyarrow

!nvidia-smi
!head /proc/cpuinfo
!head -n 3 /proc/meminfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Sun Aug  7 07:24:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                 

In [3]:
from google.colab import drive
drive.mount("/content/drive")

import os
import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pyarrow as pa
from fastpivot import pivot_table
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()

print("Usable Core : ", num_cores)
print("Pandas Version : ", pd.__version__) # 1.3.5
print("Pyarrow Version : ", pa.__version__) # 6.0.1

path = "/content/drive/MyDrive/lotte/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Usable Core :  2
Pandas Version :  1.3.5
Pyarrow Version :  6.0.1


# 데이터 생성


In [5]:
def make_datetime(df, drop=True):
    df["datetime"] = df["de_dt"].apply(str).str[:4] \
                                    + "-" + df["de_dt"].apply(str).str[4:6] \
                                    + "-" + df["de_dt"].apply(str).str[6:8] \
                                    + " " + df["de_hr"].apply(lambda x: str(x).zfill(2))
    df["datetime"] = pd.to_datetime(df["datetime"])
    if drop : df = df.drop(["de_dt", "de_hr"], axis=1)
    return df


def class_mapping(df, col, verbose=True):
    class_map = {lbl:idx for idx, lbl in enumerate(sorted(df[col].unique()))}
    if verbose : print(class_map)
    return df[col].map(class_map)

In [6]:
format_df = pd.read_parquet(path+"/LPOINT_BIG_COMP_PDDE_CUST.parquet", columns=["cust", "rct_no", "de_dt", "de_hr"], engine="pyarrow").drop_duplicates("rct_no", keep="first")
format_df = make_datetime(format_df, drop=True)
# format_df = format_df.set_index("datetime").sort_values("datetime").reset_index().set_index("rct_no") # 뒤에서 어차피 순서 바뀌어서 하지 않음..
format_df = format_df.set_index("rct_no")
print(format_df.head(5))


pdde_cust_df = pd.read_parquet(path+"/LPOINT_BIG_COMP_PDDE_CUST.parquet", columns=["rct_no", "ma_fem_dv", "ages", "chnl_dv", "zon_hlv", "cop_c"], engine="pyarrow").drop_duplicates("rct_no", keep="first").set_index("rct_no")
pdde_cust_df["ma_fem_dv"] = class_mapping(pdde_cust_df, "ma_fem_dv")
pdde_cust_df["chnl_dv"] = class_mapping(pdde_cust_df, "chnl_dv")
pdde_cust_df["ages"] = class_mapping(pdde_cust_df, "ages")
print(pdde_cust_df.head(5))



format_df = format_df.join(pdde_cust_df)
format_df["month"] = format_df.datetime.dt.month
format_df["dayofweek"] = format_df.datetime.dt.dayofweek
format_df = pd.get_dummies(format_df,
                           columns=["ages", "zon_hlv", "cop_c", "month", "dayofweek"])

format_df.head(5)

                    cust            datetime
rct_no                                      
A01000001113  M430112881 2021-01-01 10:00:00
A01000003148  M430112881 2021-01-01 10:00:00
A01000004946  M430112881 2021-01-01 10:00:00
A01000005297  M430112881 2021-01-01 10:00:00
A01000006470  M430112881 2021-01-01 11:00:00
{'남성': 0, '여성': 1}
{1: 0, 2: 1}
{'20대': 0, '30대': 1, '40대': 2, '50대': 3, '60대': 4, '70대': 5}
              ma_fem_dv  ages  chnl_dv zon_hlv cop_c
rct_no                                              
A01000001113          1     3        0     Z17   A01
A01000003148          1     3        0     Z17   A01
A01000004946          1     3        0     Z17   A01
A01000005297          1     3        0     Z17   A01
A01000006470          1     3        0     Z17   A01


,cust,datetime,ma_fem_dv,chnl_dv,ages_0,ages_1,ages_2,ages_3,ages_4,ages_5,...,month_10,month_11,month_12,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
rct_no,,,,,,,,,,,,,,,,,,,,,
A01000001113,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
A01000003148,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
A01000004946,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
A01000005297,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
A01000006470,M430112881,2021-01-01 11:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
clac_mcls_df = pd.read_parquet(path+"/LPOINT_BIG_COMP_PDDE_CUST.parquet", columns=["rct_no", "clac_mcls_nm", "buy_ct"],engine="pyarrow")
clac_mcls_df = pivot_table(clac_mcls_df, index="rct_no", columns="clac_mcls_nm", values="buy_ct", aggfunc='sum', fill_value=0).astype("uint16")
clac_mcls_df.head(5)

clac_mcls_nm,DVD/홈시어터,TV,가공유,가방세트,간식/안주형과자,간편조리,갑각류,거실가구,건강가전,건강관리용품,...,핸드/풋케어,햄/소시지,향수,향신료,헤어액세서리,헤어케어,호주산소고기,홈웨어,홍삼/인삼가공식품,화장지/티슈
rct_no,,,,,,,,,,,,,,,,,,,,,
A01000001113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A01000002265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A01000003148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A01000004946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A01000005297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 저장
processed_df = format_df.reset_index().set_index("rct_no").join(clac_mcls_df)
processed_df.to_parquet(path + "LPOINT_BIG_COMP_CLAC_MCLS_SUM.parquet", index=True, engine="pyarrow", compression=None)

In [9]:
# 로드
processed_df = pd.read_parquet(path + "LPOINT_BIG_COMP_CLAC_MCLS_SUM.parquet", engine="pyarrow")
processed_df = processed_df.reset_index().set_index("cust")
processed_df.head(3)

,rct_no,datetime,ma_fem_dv,chnl_dv,ages_0,ages_1,ages_2,ages_3,ages_4,ages_5,...,핸드/풋케어,햄/소시지,향수,향신료,헤어액세서리,헤어케어,호주산소고기,홈웨어,홍삼/인삼가공식품,화장지/티슈
cust,,,,,,,,,,,,,,,,,,,,,
M430112881,A01000001113,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
M430112881,A01000003148,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
M430112881,A01000004946,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1266589 entries, M430112881 to M866900192
Columns: 401 entries, rct_no to 화장지/티슈
dtypes: datetime64[ns](1), int64(2), object(1), uint16(349), uint8(48)
memory usage: 949.4+ MB


In [11]:
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [12]:
custs = processed_df.index.unique()
features = [col for col in format_df.columns if col not in ["rct_no", "cust", "datetime"]] + ["date_interval", "date_from"] + list(clac_mcls_df.columns)
print(*custs[:400])
print(*features)

M430112881 M646853852 M669384418 M886923095 M707991383 M117855953 M641274652 M625839441 M284446967 M401266932 M860973355 M686795542 M878181490 M704866277 M942878878 M174599398 M651841347 M577884977 M511106441 M531271959 M422047369 M234266219 M049345995 M497989990 M009032626 M931922533 M165312283 M996518031 M705629628 M184969427 M741760270 M243577483 M558692172 M377967939 M838446773 M506355556 M879094374 M732600894 M856124318 M263574190 M157204685 M650733020 M017256273 M641186030 M986125865 M232573809 M268811651 M162681947 M163175113 M440960058 M502480776 M191537184 M030477840 M690898061 M506965359 M725878700 M840551273 M746428121 M334943764 M561787985 M982559329 M807374636 M676262214 M988646555 M530294706 M900002349 M694209751 M123475113 M318802112 M038801652 M517637921 M539562667 M615380326 M609491987 M259613010 M338310215 M156930891 M419249862 M474747127 M624588650 M083434555 M049216782 M897579293 M316171414 M583031784 M370732403 M336360825 M835554931 M456838034 M586590453 M618317251

In [13]:
# def make_rct_tensor_by_cust(cust_idx, features=features, window=20, slide=10, dtype=torch.int16, drop=True):
#     global processed_df

#     cust_df = processed_df.loc[cust_idx]
#     rct_tensor_by_cust = []
#     rct_nos =[]
#     rct_length = len(cust_df)

#     if rct_length==1 or not isinstance(cust_df, pd.core.frame.DataFrame): # 길이가 1인 시리즈는 None 반환
#         return None

#     # cust index는 버려진다. rct_no도 index가 되면서 버려질 것.
#     # 구입일 간격에 대한 feature들을 생성하기 위해 sorting한다.
#     cust_df = cust_df.set_index("datetime").sort_values("datetime") \
#                  .reset_index().set_index("rct_no")
#     cust_df["date_interval"] = (cust_df.datetime - cust_df.datetime.shift(-1)).dt.days.fillna(-1) # 몇일만에 새로운 구입을 하였는지

#     cust_rcts = list(range(rct_length))
#     for i in cust_rcts[::slide]:
#         if rct_length-1 <= i: # 고객의 남은 영수증이 1개 이하인 경우 break
#             break
#         else:
#             temp = cust_df.iloc[i : i+window]
#             temp["date_from"] = (temp.datetime.iloc[0] - temp.datetime).dt.days # 최근 구입일로부터 몇일만에 구입한 영수증인지.  ex) [2]: 01/22, [1]: 02/11, [0]: 02/13 
#                                                                                                                                 # => [2]: 22,    [1]: 20,    [0]: 0
#             if not drop:
#                 rct_no = list(temp.index)
#                 rct_nos.append(rct_no)

#             temp = temp[features] # 피쳐 집합으로 원하는 피쳐를 골라 저장
#             temp = torch.tensor(temp.values, dtype=dtype) # window x features

#             if 1 < temp.shape[0] < window: # 고객의 남은 영수증이 2개 이상, 지정된 "최근 영수증 개수" 미만인 경우
#                 ones = torch.ones((window-temp.shape[0], temp.shape[1]), dtype=dtype) # "영수증 텐서" 생성, 부족한 영수증 데이터는 -1로 채움.
#                 temp = torch.cat((temp, -ones), dim=0)

#             rct_tensor_by_cust.append(temp)

        
#     rct_tensor_by_cust = torch.stack(rct_tensor_by_cust, dim=0) # cust's rcts x window x features
#     if not drop:
#         return rct_tensor_by_cust, rct_nos
#     else:
#         return rct_tensor_by_cust

In [14]:
def make_rct_tensor_by_cust(cust_idx, features=features, window=10, slide=5, dtype=torch.int16, drop=True):
    global processed_df

    cust_df = processed_df.loc[cust_idx]
    rct_tensor_by_cust = []
    rct_nos =[]
    rct_length = len(cust_df)

    if rct_length < window or not isinstance(cust_df, pd.core.frame.DataFrame): # 길이가 1인 시리즈는 None 반환
        return None

    # cust index는 버려진다. rct_no도 index가 되면서 버려질 것.
    # 구입일 간격에 대한 feature들을 생성하기 위해 sorting한다.
    cust_df = cust_df.set_index("datetime").sort_values("datetime") \
                 .reset_index().set_index("rct_no")
    cust_df["date_interval"] = (cust_df.datetime - cust_df.datetime.shift(-1)).dt.days.fillna(-1) # 몇일만에 새로운 구입을 하였는지

    cust_rcts = list(range(rct_length))
    for i in cust_rcts[::slide]:
        if rct_length-window < i: # 고객의 남은 영수증이 window개 이하인 경우 break
            break
        else:
            temp = cust_df.iloc[i : i+window]
            temp["date_from"] = (temp.datetime.iloc[0] - temp.datetime).dt.days # 최근 구입일로부터 몇일만에 구입한 영수증인지.  ex) [2]: 01/22, [1]: 02/11, [0]: 02/13 
                                                                                                                                # => [2]: 22,    [1]: 20,    [0]: 0
            if not drop:
                rct_no = list(temp.index)
                rct_nos.append(rct_no)

            temp = temp[features] # 피쳐 집합으로 원하는 피쳐를 골라 저장
            temp = torch.tensor(temp.values, dtype=dtype) # window x features

            rct_tensor_by_cust.append(temp)

        
    rct_tensor_by_cust = torch.stack(rct_tensor_by_cust, dim=0) # cust's rcts x window x features
    if not drop:
        return rct_tensor_by_cust, rct_nos
    else:
        return rct_tensor_by_cust

In [15]:
# cust_idx = custs[0]
# tens, nos = make_rct_tensor_by_cust(cust_idx, drop=False)
# nos[0]

# processed_df.loc[cust_idx].sort_values(by="datetime")[:20].rct_no

In [16]:
len(custs)

26917

In [17]:
# window = 20 # 한 텐서 안에 들어갈 "최근 영수증 개수"
# slide = 10 # 한 고객의 영수증 기록에서 window로 스캔 시의 간격
# chunk_num = 10
# sample_size = 1000

In [18]:
window = 10 # 한 텐서 안에 들어갈 "최근 영수증 개수"
slide = 5 # 한 고객의 영수증 기록에서 window로 스캔 시의 간격
chunk_num = 16
sample_size = 26912 # 26912
# rct_tensors = []
# sampled_custs = np.random.choice(list(custs), size=sample_size)

# for j in tqdm(range(sample_size//chunk_num), position=0, desc="making_data...", leave=None):
#     if j == sample_size//chunk_num-1:
#         rct_chunk_tensors = Parallel(n_jobs=num_cores)(delayed(make_rct_tensor_by_cust)(cust_idx, features=features, window=window, slide=10, dtype=torch.int16) for cust_idx in sampled_custs[j*chunk_num:])
#     else:
#         rct_chunk_tensors = Parallel(n_jobs=num_cores)(delayed(make_rct_tensor_by_cust)(cust_idx, features=features, window=window, slide=10, dtype=torch.int16) for cust_idx in sampled_custs[j*chunk_num : (j+1)*chunk_num])
#     rct_chunk_tensors = [t for t in rct_chunk_tensors if t is not None] # None을 없앤다.
#     rct_chunk_tensors = torch.cat(rct_chunk_tensors, dim=0)

#     print(f"{j}번째 chunk의 Tensor Shape : ", rct_chunk_tensors.shape)
#     rct_tensors.append(rct_chunk_tensors) # 전체 tensor에 추가
#     del rct_chunk_tensors

# rct_tensors = torch.cat(rct_tensors, dim = 0)
# rct_tensors.shape

# 모델링

In [19]:
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [20]:
# torch.save(rct_tensors, path + "LPOINT_BIG_COMP_RCT_TENSORS.pt")
rct_tensors = torch.load(path + "LPOINT_BIG_COMP_RCT_TENSORS8000.pt")

In [ ]:
rct_tensors.shape

In [61]:
class CustomDataset(Dataset):
    def __init__(self, rct_tensors = None, cust_idx = None, train_mode=True, transforms = None):
        self.transforms = transforms
        self.rct_tensors = rct_tensors
        self.train_mode = train_mode
        if not self.train_mode:
            if (make_rct_tensor_by_cust(cust_idx, drop=False) is None):
                self.tensors = []
            else:
                self.tensors, self.rct_nos = make_rct_tensor_by_cust(cust_idx, drop=False)

    def __len__(self):
        if self.train_mode :
            return self.rct_tensors.shape[0]
        else: 
            return len(self.tensors)

    def __getitem__(self, idx):
        
        if self.train_mode:
            input = self.rct_tensors[idx]
            if self.transforms is not None:
                input = self.transforms(input)[0]
            return input, input[:,52:]
        else:
            return self.tensors[idx], self.rct_nos[idx][0]


In [22]:
class Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = (
            embedding_dim, 2 * embedding_dim
        )
        self.rnn1 = nn.LSTM(
          input_size=n_features,
          hidden_size=self.hidden_dim,
          num_layers=1,
          batch_first=True
        )
        self.rnn2 = nn.LSTM(
          input_size=self.hidden_dim,
          hidden_size=embedding_dim,
          num_layers=1,
          batch_first=True
        )
    def forward(self, x):
        x, (_, _) = self.rnn1(x)
        x, (hidden_n, _) = self.rnn2(x)
        return  x[:,-1,:]

In [23]:
class Decoder(nn.Module):
    def __init__(self, input_dim=64, n_features=1):
        super(Decoder, self).__init__()
        self.dropout_prob = 0.2
        self.decoder_lin = nn.Sequential(
            nn.Linear(input_dim, 128, bias = True),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Dropout(self.dropout_prob),
            
            nn.Linear(128, 512, bias = True),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Dropout(self.dropout_prob),

            nn.Linear(512, 1024, bias = True),
            nn.BatchNorm1d(1024),
            nn.ReLU(True),
            nn.Dropout(self.dropout_prob),

            nn.Linear(1024, (n_features-52) * window, bias = True),
            nn.ReLU(True)
        )
        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(window, n_features - 52))
        
        
    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x) - 1

        return x

In [24]:
class RecurrentAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(RecurrentAutoencoder, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim)#.to(device)
        self.decoder = Decoder(embedding_dim, n_features)#.to(device)

    def forward(self, x, train_mode = True):
        if train_mode:
            x = self.encoder(x)
            x = self.decoder(x)
        else:
            x = self.encoder(x)
        return x

In [25]:
from torchvision.transforms import Lambda

def lambda_function(x):
    y = x[:, :, 50:]
    y = torch.log1p(y)
    y = torch.nn.functional.normalize(y, dim=1)
    x = torch.cat((x[:, :, :50], y), dim = 2)
    return x

transform = transforms.Compose([
    # 0~1의 범위를 가지도록 정규화
    transforms.ToTensor(),
    # transforms.Normalize((0.485), (0.229))
    ])

# transform = transforms.Compose([
#     Lambda(lambda_function)
#     ])

window = 10 # 한 텐서 안에 들어갈 "최근 영수증 개수"
slide = 5 # 한 고객의 영수증 기록에서 window로 스캔 시의 간격

In [26]:
#n X n_features  시계열 데이터를 1 X 128 형태로 변환
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = RecurrentAutoencoder(window, 401, 10)
model = model.to(device)

batch_size = 512

In [27]:
train_id, valid_id = train_test_split(range(rct_tensors.shape[0]), test_size = 0.4, random_state = 42)
valid_id, test_id = train_test_split(valid_id, test_size = 0.5, random_state = 42)

In [28]:
# log를 취해도 np.uint8 떄문에 소용 없어진다.
train_dataset = CustomDataset(rct_tensors.detach().numpy().astype(np.uint8)[train_id], 
                              train_mode = True, transforms = transform)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False, num_workers=0)

valid_dataset = CustomDataset(rct_tensors.detach().numpy().astype(np.uint8)[valid_id], 
                              train_mode = True, transforms = transform)
valid_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False, num_workers=0)

In [24]:
rct_tensors.detach().numpy().astype(np.uint8)[valid_id].shape

(6981, 10, 401)

In [29]:
for idx, batch in enumerate(valid_loader):
    x, y = batch[0].float().to(device), batch[1].float().to(device)
    break

In [35]:
np.max(x.detach().numpy())

TypeError: ignored

In [30]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for idx, batch in enumerate(vali_loader):
            x, y = batch[0].float().to(device), batch[1].float().to(device)

            pred = model(x)
            loss = criterion(pred, y)
            
            vali_loss.append(loss.item()/batch_size)

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# Loss Function
criterion = nn.L1Loss(reduction='sum').to(device)

In [32]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):

    best_mae = 9999
    
    for epoch in range(500):
        model.train()
        train_loss = []
        for idx, batch in tqdm(enumerate(train_loader)):
            x, y = batch[0].float().to(device), batch[1].float().to(device)
            optimizer.zero_grad()
            # Data -> Model -> Output
            pred = model(x)
            
            # Calc loss
            loss = criterion(pred, y)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item()/batch_size)
                        
        # Evaluation Validation set
        vali_mae = validation(model, valid_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')

In [33]:
train(model, optimizer, train_loader,valid_loader, None, device)

41it [00:02, 15.08it/s]


Epoch [0] Train MAE : [775.51662] Validation MAE : [486.44196]



41it [00:01, 24.63it/s]


Epoch [1] Train MAE : [358.85300] Validation MAE : [190.51990]



41it [00:01, 24.91it/s]


...

Epoch [58] Train MAE : [170.54222] Validation MAE : [125.27957]



41it [00:01, 25.25it/s]


Epoch [59] Train MAE : [165.78089] Validation MAE : [203.62300]



41it [00:01, 25.43it/s]


KeyboardInterrupt: ignored

In [53]:
cust_idx = custs[12]  #1234가 안되는거
test_dataset = CustomDataset(rct_tensors = None, cust_idx = cust_idx, train_mode = False, transforms = None)
test_dataset.__len__()

6

In [66]:
cust_idx = custs[145]  #1234가 안되는거
test_dataset = CustomDataset(rct_tensors = None, cust_idx = cust_idx, train_mode = False, transforms = None)
test_dataset.__len__()

23

In [55]:
custs[144]

'M411045461'

In [73]:
df = pd.DataFrame()
for i in tqdm(range(26917)):    
    cust_idx = custs[i]  #1234가 안되는거
    test_dataset = CustomDataset(rct_tensors = None, cust_idx = cust_idx, train_mode = False, transforms = None) # 어떤 고객은 데이터셋이 없을 수 있음.
    if test_dataset.__len__() == 0:
        continue
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=0)

    model.eval() # Evaluation
    results = pd.DataFrame()
    with torch.no_grad():
        for idx, batch in enumerate(test_loader):
            x, rct_no = batch[0].float().to(device), batch[1]

            pred = model(x, train_mode = False).to("cpu").detach().numpy()
            result = pd.DataFrame(pred)
            result = pd.concat((pdde_cust_df.loc[list(rct_no)].reset_index(), result), axis=1)
            
            results = pd.concat((results, result), axis=0)
            # break

    df = pd.concat((df, result), axis=0)
df

100%|██████████| 26917/26917 [43:38<00:00, 10.28it/s]


,rct_no,ma_fem_dv,ages,chnl_dv,zon_hlv,cop_c,0,1,2,3,4,5,6,7,8,9
0,A01000001113,1,3,0,Z17,A01,-0.003499,-0.097034,0.132439,0.090437,-0.081073,-0.096178,0.020280,-0.243195,-0.447459,0.026381
1,A01000007648,1,3,0,Z17,A01,-0.161823,-0.124157,0.189549,0.019463,-0.061196,0.038573,0.098080,-0.238949,-0.348969,0.163122
2,A03104921319,1,3,0,Z17,A03,-0.152364,-0.111995,0.171075,-0.058756,-0.041543,0.005752,0.120717,-0.219868,-0.379981,0.123004
3,A03104969721,1,3,0,Z17,A03,-0.153275,-0.116798,0.176958,-0.126616,-0.038817,0.008592,0.106148,-0.223084,-0.345886,0.144454
4,A03089375637,1,3,0,Z17,A03,-0.122719,-0.127713,0.144397,-0.048440,-0.052536,0.040808,0.115069,-0.230942,-0.293541,0.172711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,E06043646151,1,2,1,Z17,A06,-0.167120,-0.086208,0.172319,-0.144060,-0.000482,0.018715,0.133233,-0.149462,-0.167701,0.171611
5,E06045000396,1,2,1,Z17,A06,-0.149323,-0.135856,0.123108,-0.000361,-0.053221,0.039866,0.095177,-0.219620,-0.196483,0.188880
6,E06047254820,1,2,1,Z17,A06,-0.155091,-0.109149,0.132143,-0.035873,-0.043597,0.047228,0.100373,-0.220499,-0.195892,0.171555
0,E06043099892,0,2,1,Z10,A06,-0.139966,-0.126596,0.141192,0.003646,-0.057350,0.046370,0.093213,-0.208389,-0.196228,0.199740


In [76]:
df.to_csv(path + 'df.csv',index=False)